In [63]:
from pool import AMMPool, AMMPoolHistory, CPMM

In [64]:
import pandas as pd 

**Define Pools to study and initialize**

In [65]:
# Define token0=USDC and token1=WETH
token0 = '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'
token1 = '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'

**Get Uni trades stream**  

In [66]:
from datastreams.datastream import Streamer

query_size = 10000

# instantiate Streamer object. 
# Note - unlike the cow queries, univ3 does not require multiple streamer instantations because the swaps field path is reset each iteration. 
# If the Cow queries were updated to use the same method, we could use the same streamer object for all queries.
univ3_ds = Streamer('https://api.thegraph.com/subgraphs/name/messari/uniswap-v3-ethereum')

# get a query field path from the query dictionary which is automatically populated in the Streamer object
uni_swaps_fp = univ3_ds.queryDict.get('swaps')

# define query path and parameters 
uni_swaps_qp = uni_swaps_fp(
        first=query_size, 
        orderBy='timestamp',
        orderDirection='asc',
        where = {'pool': '0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8'} 
        )

# run query
uni_swaps_df = univ3_ds.runQuery(uni_swaps_qp)

FIELD - swaps


In [67]:
uni_swaps_df.shape

(10000, 20)

In [68]:
uni_swaps_df.dtypes

swaps_id               object
swaps_hash             object
swaps_nonce             int64
swaps_logIndex          int64
swaps_gasLimit          int64
swaps_gasUsed          object
swaps_gasPrice          int64
swaps_protocol_id      object
swaps_account_id       object
swaps_pool_id          object
swaps_blockNumber       int64
swaps_timestamp         int64
swaps_tick              int64
swaps_tokenIn_id       object
swaps_amountIn         object
swaps_amountInUSD     float64
swaps_tokenOut_id      object
swaps_amountOut        object
swaps_amountOutUSD    float64
endpoint               object
dtype: object

In [69]:
uni_swaps_df.head(5)

,swaps_id,swaps_hash,swaps_nonce,swaps_logIndex,swaps_gasLimit,swaps_gasUsed,swaps_gasPrice,swaps_protocol_id,swaps_account_id,swaps_pool_id,swaps_blockNumber,swaps_timestamp,swaps_tick,swaps_tokenIn_id,swaps_amountIn,swaps_amountInUSD,swaps_tokenOut_id,swaps_amountOut,swaps_amountOutUSD,endpoint
0,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,0xce7c3c307d820785caa12938012372fc9366a614a6aa...,1,26,193389,None,41000000000,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x5eefc9306f11a824762ccdaedac41049efc7fcc8,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,12371376,1620179783,195285,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100000000000000,0.000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,329608,3.296080e-01,uniswap-v3-ethereum
1,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,0x9a1c51b0bffbf840948f3b6e3f3e495ba1cd3fa64854...,14,192,262131,None,46000000000,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x0459b3fbf7c1840ee03a63ca4aa95de48322322e,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,12373132,1620203006,195298,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,50000000000000000,165.300088,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,164694492,1.646945e+02,uniswap-v3-ethereum
2,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,0xc58715c62a5bf70a6ca09f0e51546d6cad76c8d4fff0...,622,8,300000,None,62000000000,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xfb36693ac2dbe8dfc9f89dd0de6015c6ea66b0bf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,12373520,1620208251,195298,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,100000000000000,0.330160,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,329169,3.291690e-01,uniswap-v3-ethereum
3,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,0x288c21b8b4fbf449b1d086a06e43b124ac2bc088c3f5...,628,86,500000,None,37180000302,0x1f98431c8ad98523631ae4a59f267346ea31f984,0xfb36693ac2dbe8dfc9f89dd0de6015c6ea66b0bf,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,12374077,1620215997,195298,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2,0.000002,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,329169,1.086782e-09,uniswap-v3-ethereum
4,0x67502d8ba373287f6d301f6baa77c5a5f4c80d0753c3...,0x67502d8ba373287f6d301f6baa77c5a5f4c80d0753c3...,65,257,711171,None,40000000000,0x1f98431c8ad98523631ae4a59f267346ea31f984,0x33a68cdc3e3b4151640e260dc4436fed7d1f4c05,0x8ad599c3a0ff1de082011efddc58f1908eb6e6d8,12374320,1620219416,195173,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,1559137299,1559.137299,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,467880854065813753,1.544752e+03,uniswap-v3-ethereum


**1.Simple AMM. SWAP ALL** 

In [70]:
# Create an instance of AMMPoolHistory 
feedlot1 = AMMPoolHistory(100000000000000000000000, 666666666666666666666666)

In [71]:
# Iterate through the rows of the dataframe
for index, row in uni_swaps_df.iterrows():
    # Determine if the swap is a buy or a sell
    token_in = row['swaps_tokenIn_id']
    token_out = row['swaps_tokenOut_id']
    is_buy = token_in == token0 and token_out == token1
    
    # Extract the amountIn, amountOut, and timestamp
    if is_buy:
        amt0 = row['swaps_amountIn']
        amt1 = row['swaps_amountOut']
    else:
        amt0 = row['swaps_amountOut']
        amt1 = row['swaps_amountIn']
    ts = row['swaps_timestamp']
    
    print('amt0', amt0)
    print('amt1', amt1)
    print('ts', ts)
    
    # Run the swap function on the AMM pool
    feedlot1.swap(amt0, amt1, ts)

amt0 329608
amt1 100000000000000
ts 1620179783
amt0 164694492
amt1 50000000000000000
ts 1620203006
amt0 329169
amt1 100000000000000
ts 1620208251
amt0 2
amt1 329169
ts 1620215997
amt0 1559137299
amt1 467880854065813753
ts 1620219416
amt0 999956
amt1 300000000000000
ts 1620238163
amt0 9994340899
amt1 2943826520162862564
ts 1620238435
amt0 34176041
amt1 10000000000000000
ts 1620238934
amt0 2252650764
amt1 660000000000000000
ts 1620240052
amt0 34062094
amt1 10000000000000000
ts 1620240161
amt0 1020463340
amt1 300000000000000000
ts 1620240206
amt0 3997268559
amt1 1164863556126674691
ts 1620240686
amt0 1000293367
amt1 292651814412636650
ts 1620240799
amt0 155000340
amt1 45374124000000000
ts 1620240859
amt0 85390993
amt1 25000000000000000
ts 1620240866
amt0 55078592
amt1 16126543385509870
ts 1620241083
amt0 2250430202
amt1 660000000000000000
ts 1620241124
amt0 679676025
amt1 200000000000000000
ts 1620241167
amt0 339400775
amt1 100000000000000000
ts 1620241269
amt0 1003273875
amt1 29598427166

ts 1620489989
amt0 198937437591
amt1 53750000000000000000
ts 1620490093
amt0 199079622623
amt1 53629500000000000000
ts 1620490150
amt0 99999999999
amt1 26978849259968055051
ts 1620490150
amt0 133849787576
amt1 35996167608483265163
ts 1620490269
amt0 155384889088
amt1 41725747331632327597
ts 1620490452
amt0 198795542968
amt1 53286100000000000000
ts 1620490500
amt0 269580413168
amt1 72084333704811959042
ts 1620490581
amt0 21960888831
amt1 5900000000000000000
ts 1620490740
amt0 59687313618
amt1 15936366410332760087
ts 1620491692
amt0 152602393057
amt1 40699478523678884428
ts 1620491774
amt0 199005231521
amt1 52978200000000000000
ts 1620491910
amt0 199066175843
amt1 52884500000000000000
ts 1620492037
amt0 17462144692
amt1 4662587220166473807
ts 1620492172
amt0 198919336182
amt1 52745600000000000000
ts 1620492186
amt0 7524034219
amt1 2000000000000000000
ts 1620492188
amt0 266769935607
amt1 70565368392533449863
ts 1620492188
amt0 22678252807
amt1 6027262131044375238
ts 1620492188
amt0 283081

amt0 598128967720
amt1 148369200000000000000
ts 1620748978
amt0 598240343322
amt1 147880200000000000000
ts 1620749113
amt0 200000000000
amt1 49678750239546711298
ts 1620749142
amt0 41262027763
amt1 10256445770830655584
ts 1620749246
amt0 542818793100
amt1 133925304472287980555
ts 1620749275
amt0 100020852855
amt1 24631547665908324683
ts 1620749551
amt0 291647835961
amt1 71740698223853584200
ts 1620749555
amt0 50000000000
amt1 12286995040541589962
ts 1620749649
amt0 50000000000
amt1 12283426876683436493
ts 1620749723
amt0 50000000000
amt1 12279860266900735731
ts 1620749784
amt0 50000000000
amt1 12276295210291137509
ts 1620749861
amt0 50000000000
amt1 12272731705952946488
ts 1620749905
amt0 100000000000
amt1 24534779103472396983
ts 1620750003
amt0 100000000000
amt1 24520543690862908229
ts 1620750092
amt0 220000000000
amt1 54289339088387842567
ts 1620750377
amt0 78000000000
amt1 19140645185486746753
ts 1620750430
amt0 601772386256
amt1 148787700000000016384
ts 1620750737
amt0 602288589656

amt0 326624918142
amt1 88335966946089642867
ts 1620928916
amt0 394110000000
amt1 106204617404795167816
ts 1620928934
amt0 338236902083
amt1 91319400000000000000
ts 1620928934
amt0 445783863658
amt1 119879632416536610886
ts 1620928978
amt0 14539051334
amt1 3929205470000000000
ts 1620929068
amt0 5180100464
amt1 1400000000000000000
ts 1620929112
amt0 11099768287
amt1 3000000000000000000
ts 1620929116
amt0 463530167105
amt1 125428295646935665637
ts 1620929124
amt0 692389116539
amt1 186129197831966176557
ts 1620929139
amt0 362500000000
amt1 97204951464139739809
ts 1620929166
amt0 21330997678
amt1 5759253782746333494
ts 1620929166
amt0 19920683939
amt1 5369425060000000000
ts 1620929283
amt0 901334491201
amt1 243500000000000000000
ts 1620929401
amt0 4284225718
amt1 1160000000000000000
ts 1620929563
amt0 111561431677
amt1 30215118581196621296
ts 1620929709
amt0 30500000000
amt1 8228552858757180349
ts 1620929801
amt0 378644621460
amt1 102677743128165580800
ts 1620929801
amt0 350337657793
amt1 9

amt0 19070009584
amt1 5072160455992062029
ts 1621105515
amt0 518392300632
amt1 138813246742526705664
ts 1621105709
amt0 2140063383
amt1 570064661252937726
ts 1621105715
amt0 309949658712
amt1 83100000000000000000
ts 1621105766
amt0 500603742406
amt1 134508070000000000000
ts 1621105839
amt0 311389376659
amt1 83564707259157381120
ts 1621105839
amt0 337429734275
amt1 90780000000000000000
ts 1621105885
amt0 30000000000
amt1 8026408209451930160
ts 1621105958
amt0 905193582110
amt1 243844751661764050944
ts 1621106072
amt0 163549334794
amt1 43744299819262863349
ts 1621106072
amt0 50992604985
amt1 13756521551250721141
ts 1621106099
amt0 94612570092
amt1 25529719435725645081
ts 1621106104
amt0 64548245727
amt1 17421531826009589976
ts 1621106171
amt0 379515719235
amt1 102500000000000000000
ts 1621106276
amt0 2999879765
amt1 805816272261831881
ts 1621106303
amt0 200691929273
amt1 54250273227439550000
ts 1621106328
amt0 727335578188
amt1 196887500000000000000
ts 1621106335
amt0 105898959770
amt1 2

In [72]:
results1 = pd.DataFrame(feedlot1.history)

In [73]:
results.shape

(20001, 3)

In [74]:
results.head(5)

,ts,reserve0,reserve1
0,0,100000000000000000000000,666666666666666666666666
1,1620179783,100000000000000000000119,666666666666666666337058
2,1620203006,100000000000000000059141,666666666666666501642566
3,1620208251,100000000000000000059259,666666666666666501313397
4,1620215997,100000000000000000059260,666666666666666501313395


**OraclePool AMM**

**get cow trades for oracle pricing** 

In [75]:
cow_ds = Streamer('https://api.thegraph.com/subgraphs/name/cowprotocol/cow')

# get COW trades 

trades_fp = cow_ds.queryDict.get('trades')

# trades query path that gets token a -> token b trades
trades_qp = trades_fp(
    first=10000,
    orderBy='timestamp',
    orderDirection='asc',
    where={ 'sellToken':'0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
           'buyToken':'0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'
    }
)

# run query
trades_df = cow_ds.runQuery(trades_qp)

FIELD - trades


In [76]:
trades_df.shape

(9588, 16)

In [77]:
trades_df.head(5)

,trades_id,trades_timestamp,trades_gasPrice,trades_feeAmount,trades_txHash,trades_settlement_id,trades_buyAmount,trades_sellAmount,trades_sellToken_id,trades_buyToken_id,trades_order_id,trades_buyAmountEth,trades_sellAmountEth,trades_buyAmountUsd,trades_sellAmountUsd,endpoint
0,0x964d79cc0d70cd941fff74fbfa9cbaadc7bdcb5b2df9...,1628256221,40538405964,3936941,0x94b95d1be273b4a81cd5671c8907b8052003884fa9d2...,0x94b95d1be273b4a81cd5671c8907b8052003884fa9d2...,878248156599662998,2450693880,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x964d79cc0d70cd941fff74fbfa9cbaadc7bdcb5b2df9...,0.878248,0.882615,2438.567613,2450.693880,cow
1,0x65ecbd5e612c7c1c6c2e2947cce18ece70ec9951810c...,1628256543,49102510478,5229861,0xec35b71458c23c335a502697257e4f1d9455458571c6...,0xec35b71458c23c335a502697257e4f1d9455458571c6...,994057037726850263,2775435190,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x65ecbd5e612c7c1c6c2e2947cce18ece70ec9951810c...,0.994057,0.999071,2761.507606,2775.435190,cow
2,0x88c1c3501318eb059c2d48e9521e6e810231c5daf642...,1628685444,54723085347,6735290,0xc5a29ad76c33991144132596785db164bf00939cde7f...,0xc5a29ad76c33991144132596785db164bf00939cde7f...,110649028673050746991,357460414456,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x88c1c3501318eb059c2d48e9521e6e810231c5daf642...,110.649029,110.965957,356439.476432,357460.414456,cow
3,0xa7955a8c780c1a4b7e5724b9f0d12b2acf046ed1b52c...,1628691255,76437161120,11396782,0xb29f47b414e851a4502728b236ca3b7e55b072063015...,0xb29f47b414e851a4502728b236ca3b7e55b072063015...,14230130525021492,57721266,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xa7955a8c780c1a4b7e5724b9f0d12b2acf046ed1b52c...,0.014230,0.017754,46.264277,57.721266,cow
4,0x4dda11c8857ca3706cd2b17b892b1ce86f4a7dab654f...,1628699635,115332199115,16612673,0xf7b3c927fb50b2da16d600e218da01166ba1cf7ecfd8...,0xf7b3c927fb50b2da16d600e218da01166ba1cf7ecfd8...,148697702218736537,500000000,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0x4dda11c8857ca3706cd2b17b892b1ce86f4a7dab654f...,0.148698,0.154103,482.463260,500.000000,cow


In [78]:
# Set query size large enough to retrieve all tokens in the graph
query_size = 10000

token_fp = cow_ds.queryDict.get('tokens')

# add parameters to the settlements_qp.
token_qp = token_fp(
    first=query_size,
)

# run query
token_df = cow_ds.runQuery(token_qp)


FIELD - tokens


In [79]:
token_df.shape

(2966, 13)

In [80]:
# Merge1 on sell token address.  x = sell token 

merged_df = pd.merge(trades_df, token_df, left_on='trades_sellToken_id', right_on='tokens_address')

merged_df.shape

(9588, 29)

In [81]:
# Merge2 on buy token address.  y = buy token 
merged_df2 = pd.merge(merged_df, token_df, left_on='trades_buyToken_id', right_on='tokens_address')

merged_df2.shape

(9588, 42)

In [82]:
merged_df2.dtypes

trades_id                        object
trades_timestamp                  int64
trades_gasPrice                   int64
trades_feeAmount                  int64
trades_txHash                    object
trades_settlement_id             object
trades_buyAmount                 object
trades_sellAmount                 int64
trades_sellToken_id              object
trades_buyToken_id               object
trades_order_id                  object
trades_buyAmountEth             float64
trades_sellAmountEth            float64
trades_buyAmountUsd             float64
trades_sellAmountUsd            float64
endpoint_x                       object
tokens_id_x                      object
tokens_address_x                 object
tokens_firstTradeTimestamp_x      int64
tokens_name_x                    object
tokens_symbol_x                  object
tokens_decimals_x                 int64
tokens_totalVolume_x             object
tokens_priceEth_x               float64
tokens_priceUsd_x               float64


In [83]:
# calculate buy and sell amounts from the correct decimal 
merged_df2['sell_amount'] = merged_df2.apply(lambda x: x['trades_sellAmount'] / (10**x['tokens_decimals_x']), axis=1)
merged_df2['buy_amount'] = merged_df2.apply(lambda x: x['trades_buyAmount'] / (10**x['tokens_decimals_y']), axis=1)

In [84]:
# calculate COW price defined sell amount / buy amount
merged_df2['cow_price'] = merged_df2['sell_amount'] / merged_df2['buy_amount']

**Oracle pool simulation**

In [85]:
from pool import CPMM, AMMPool, SwapRejectedError

In [86]:
# Create an instance of AMMPoolHistory 
feedlot2 = AMMPoolHistory(100000000000000000000000, 666666666666666666666666)

In [87]:
feedlot3 = OraclePool(feedlot2, oracle_dict)

In [88]:
# Iterate through the rows of the dataframe
for index, row in uni_swaps_df.iterrows():
    # Determine if the swap is a buy or a sell
    token_in = row['swaps_tokenIn_id']
    token_out = row['swaps_tokenOut_id']
    is_buy = token_in == token0 and token_out == token1
    
    # Extract the amountIn, amountOut, and timestamp
    if is_buy:
        amt0 = row['swaps_amountIn']
        amt1 = row['swaps_amountOut']
    else:
        amt0 = row['swaps_amountOut']
        amt1 = row['swaps_amountIn']
    ts = row['swaps_timestamp']
    
    print('amt0', amt0)
    print('amt1', amt1)
    print('ts', ts)
    
    # Run the swap function on the AMM pool
    feedlot3.market_order_buy(amt0, 1, ts)

amt0 329608
amt1 100000000000000
ts 1620179783
amt0 164694492
amt1 50000000000000000
ts 1620203006
amt0 329169
amt1 100000000000000
ts 1620208251
amt0 2
amt1 329169
ts 1620215997
amt0 1559137299
amt1 467880854065813753
ts 1620219416
amt0 999956
amt1 300000000000000
ts 1620238163
amt0 9994340899
amt1 2943826520162862564
ts 1620238435
amt0 34176041
amt1 10000000000000000
ts 1620238934
amt0 2252650764
amt1 660000000000000000
ts 1620240052
amt0 34062094
amt1 10000000000000000
ts 1620240161
amt0 1020463340
amt1 300000000000000000
ts 1620240206
amt0 3997268559
amt1 1164863556126674691
ts 1620240686
amt0 1000293367
amt1 292651814412636650
ts 1620240799
amt0 155000340
amt1 45374124000000000
ts 1620240859
amt0 85390993
amt1 25000000000000000
ts 1620240866
amt0 55078592
amt1 16126543385509870
ts 1620241083
amt0 2250430202
amt1 660000000000000000
ts 1620241124
amt0 679676025
amt1 200000000000000000
ts 1620241167
amt0 339400775
amt1 100000000000000000
ts 1620241269
amt0 1003273875
amt1 29598427166

amt0 570000000
amt1 149440221315758978
ts 1620502756
amt0 5128813779
amt1 1344608213085861268
ts 1620502793
amt0 80008523625
amt1 20965668206524707292
ts 1620502917
amt0 13021354871
amt1 3410382799900390592
ts 1620502989
amt0 5274957660
amt1 1381406814749177597
ts 1620503267
amt0 10403938297
amt1 2741085437700959228
ts 1620503512
amt0 274878581628
amt1 72537009015781946810
ts 1620503604
amt0 20129871323
amt1 5320805687571686500
ts 1620503604
amt0 10000000000
amt1 2643685291449165385
ts 1620503935
amt0 100000000000
amt1 26265261896543401570
ts 1620504054
amt0 100000000000
amt1 26235932483744055025
ts 1620504140
amt0 39548345190
amt1 10367794650768248531
ts 1620504263
amt0 234897647478
amt1 62018617176814436450
ts 1620504684
amt0 33673913462
amt1 8847429865149229278
ts 1620504698
amt0 101017841037
amt1 26521298534333006438
ts 1620504698
amt0 1031290159
amt1 270601358338996544
ts 1620504715
amt0 18521165069
amt1 4859258207474622489
ts 1620504874
amt0 199272110965
amt1 52218000000000000000

amt0 220000000000
amt1 54289339088387842567
ts 1620750377
amt0 78000000000
amt1 19140645185486746753
ts 1620750430
amt0 601772386256
amt1 148787700000000016384
ts 1620750737
amt0 602288589656
amt1 149437500000000016384
ts 1620751525
amt0 601595359348
amt1 149787299999999983616
ts 1620751615
amt0 152287270324
amt1 38000000000000000000
ts 1620751965
amt0 58516714834
amt1 14610513654127211064
ts 1620752482
amt0 74721896546
amt1 18663854175559216065
ts 1620754622
amt0 597709502151
amt1 148176299999999983616
ts 1620755612
amt0 163201254237
amt1 40369897341287640189
ts 1620755672
amt0 3257368378
amt1 810233474768938655
ts 1620755927
amt0 260259587327
amt1 64301089464959718804
ts 1620757048
amt0 598383258213
amt1 147473999999999983616
ts 1620757180
amt0 217700000000
amt1 53526429210545062616
ts 1620757276
amt0 315915901632
amt1 77554929663155224737
ts 1620757676
amt0 434203685852
amt1 106362220362814748446
ts 1620757706
amt0 4062933176
amt1 1000000000000000000
ts 1620757942
amt0 6094218400
am

amt0 97047289972
amt1 26250526167060621165
ts 1620907581
amt0 541060910852
amt1 147051165115626833874
ts 1620907581
amt0 676887077575
amt1 183059000000000000000
ts 1620907584
amt0 229924129920
amt1 62646893308228228229
ts 1620907584
amt0 641305710689
amt1 172570439451112129980
ts 1620907594
amt0 295832638999
amt1 80000000000000000000
ts 1620907594
amt0 120518117495
amt1 32508768343492040232
ts 1620907594
amt0 204964632517
amt1 55110859441490585018
ts 1620907619
amt0 969193457
amt1 260429524006176817
ts 1620907902
amt0 251555221227
amt1 67541670848710250935
ts 1620907905
amt0 349962436381
amt1 93787949704042377980
ts 1620907954
amt0 112724703315
amt1 30165922424022923972
ts 1620907971
amt0 943905341281
amt1 251762125973053942786
ts 1620907971
amt0 174508496567
amt1 46713919629059427306
ts 1620908015
amt0 254560148079
amt1 68233945971199237238
ts 1620908015
amt0 800000000
amt1 213322209991146508
ts 1620908074
amt0 223286898921
amt1 59498349180242241940
ts 1620908075
amt0 678388152591
amt

amt0 95671936826
amt1 24938294902680059904
ts 1621074446
amt0 114893160667
amt1 30000000000000000000
ts 1621074490
amt0 546128067738
amt1 142782665282827454079
ts 1621074494
amt0 473229203118
amt1 123972247561280644773
ts 1621074578
amt0 287753871129
amt1 74959193613449621078
ts 1621074707
amt0 41994294145
amt1 11000000000000000000
ts 1621074835
amt0 8902907954
amt1 2328725277440026563
ts 1621074853
amt0 384297655900
amt1 99992331287599338644
ts 1621074853
amt0 477560507729
amt1 125035611625299260273
ts 1621074880
amt0 588349295794
amt1 153086605445193538505
ts 1621074899
amt0 500000000000
amt1 129823547481918548615
ts 1621074943
amt0 6986303304
amt1 1823182507461690423
ts 1621075020
amt0 457592710793
amt1 119522520159841204405
ts 1621075129
amt0 38248578408
amt1 10000000000000000000
ts 1621075174
amt0 30594810660
amt1 8000000000000000000
ts 1621075367
amt0 107053100969
amt1 28000000000000000000
ts 1621075392
amt0 338851269402
amt1 88057100000000000000
ts 1621075590
amt0 54242996110
am

amt0 187672193189
amt1 54082934647480057856
ts 1621194066
amt0 25035954552
amt1 7218495358417618784
ts 1621194113
amt0 17339764638
amt1 5000000000000000000
ts 1621194113
amt0 232027229518
amt1 66946273647085281280
ts 1621194142
amt0 173124675775
amt1 50000000000000000000
ts 1621194157
amt0 34607618889
amt1 10000000000000000000
ts 1621194172
amt0 326569479299
amt1 94445620775282491392
ts 1621194240
amt0 24094781382
amt1 6974228913608930547
ts 1621194266
amt0 1137119478
amt1 329158991176460408
ts 1621194290
amt0 617507880700
amt1 179015963897140707328
ts 1621194294
amt0 34440188742
amt1 10000000000000000000
ts 1621194342
amt0 4500000000
amt1 1298879360141002817
ts 1621194359
amt0 200510091957
amt1 58251682861163950000
ts 1621194372
amt0 340179475545
amt1 99255400000000000000
ts 1621194418
amt0 340675995666
amt1 99234500000000000000
ts 1621194418
amt0 274689213590
amt1 79893971843300540416
ts 1621194418
amt0 91553896698
amt1 26741385034070077317
ts 1621194447
amt0 200550344180
amt1 586195

In [89]:
results = pd.DataFrame(feedlot.history)

In [90]:
results.shape

(20001, 3)

In [91]:
results.head(5)

,ts,reserve0,reserve1
0,0,100000000000000000000000,666666666666666666666666
1,1620179783,100000000000000000000119,666666666666666666337058
2,1620203006,100000000000000000059141,666666666666666501642566
3,1620208251,100000000000000000059259,666666666666666501313397
4,1620215997,100000000000000000059260,666666666666666501313395
